In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 46.4 MB/s 
     |████████████████████████████████| 3.3 MB 35.3 MB/s 
     |████████████████████████████████| 636 kB 51.3 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import re
from textblob import TextBlob

path = "/content/gdrive/MyDrive/enter_weights/"
df = pd.read_csv(path+"clean_train.csv")
df_test = pd.read_csv(path+"clean_test.csv")
# df = df.dropna()

df.head()

,text,aspect,label,clean_text,clean_aspect
0,can you check whether its cancelled completely?,cancelled,1,can you check whether its canceled completely?,canceled
1,cannot rely on both milk delivery and grocery ...,Milk,0,cannot rely on both milk delivery and grocer i...,milk
2,"I get no notification, however the app is real...",notification,0,"i get no ratification, however the pp is reall...",ratification
3,"Love this app, but would love it even more if ...",view,1,"love this pp, but would love it even more if w...",view
4,it does not let me load a clip on the scene,load,0,it does not let me load a clip on the scene,load


In [ ]:
punctuation = '!"#$%()*+-/:;<=>@[\\]^_`{|}~'

df["clean_text"] = df["text"].apply(lambda x: re.sub(r"http\S+", "", x))
df["clean_text"] = df["clean_text"].apply(lambda x: "".join(ch for ch in x if ch not in set(punctuation)))
df["clean_text"] = df["clean_text"].str.lower()
df["clean_text"] = df["clean_text"].str.replace("[0-9]", " ")
df["clean_text"] = df["clean_text"].apply(lambda x:" ".join(x.split()))
# df["clean_text"] = df["clean_text"].apply(lambda x: str(TextBlob(x).correct()))


df["clean_aspect"] = df["aspect"].apply(lambda x: re.sub(r"http\S+", "", x))
df["clean_aspect"] = df["clean_aspect"].apply(lambda x: "".join(ch for ch in x if ch not in set(punctuation)))
df["clean_aspect"] = df["clean_aspect"].str.lower()
df["clean_aspect"] = df["clean_aspect"].str.replace("[0-9]", " ")
df["clean_aspect"] = df["clean_aspect"].apply(lambda x:" ".join(x.split()))
# df["clean_aspect"] = df["clean_aspect"].apply(lambda x: str(TextBlob(x).correct()))


df_test["clean_text"] = df_test["text"].apply(lambda x: re.sub(r"http\S+", "", x))
df_test["clean_text"] = df_test["clean_text"].apply(lambda x: "".join(ch for ch in x if ch not in set(punctuation)))
df_test["clean_text"] = df_test["clean_text"].str.lower()
df_test["clean_text"] = df_test["clean_text"].str.replace("[0-9]", " ")
df_test["clean_text"] = df_test["clean_text"].apply(lambda x:" ".join(x.split()))
# df_test["clean_text"] = df_test["clean_text"].apply(lambda x: str(TextBlob(x).correct()))


df_test["clean_aspect"] = df_test["aspect"].apply(lambda x: re.sub(r"http\S+", "", x))
df_test["clean_aspect"] = df_test["clean_aspect"].apply(lambda x: "".join(ch for ch in x if ch not in set(punctuation)))
df_test["clean_aspect"] = df_test["clean_aspect"].str.lower()
df_test["clean_aspect"] = df_test["clean_aspect"].str.replace("[0-9]", " ")
df_test["clean_aspect"] = df_test["clean_aspect"].apply(lambda x:" ".join(x.split()))

# df_test["clean_aspect"] = df_test["clean_aspect"].apply(lambda x: str(TextBlob(x).correct()))


# df.to_csv(path+"clean_train.csv", index=False)
# df_test.to_csv(path+"clean_test.csv", index=False)

In [ ]:
import numpy as np


def build_k_datasets(texts, phrases, targets, splits=5, random_state=0):
    """
    Parameters
    ----------
    texts: np.array[(str)] or List[str]
      A array of texts
    phrases: np.array[(str)] or List[str]
      A array of phrases
    splits: int
      Number of splits
    random_state: int
      Random state for reprodubility

    Return
    ------
    train_dataset: List[List[str]]
        A k fold train split of strings in this format (Texts_array, Phrase_array, targets_array)
    test_dataset: List[List[str]]
        A k fold test split of strings in this format (Texts_array, Phrase_array, targets_array)
    """
    from sklearn.model_selection import StratifiedKFold

    skf = StratifiedKFold(n_splits=splits, shuffle=True,
                          random_state=random_state)

    if type(texts) != np.ndarray:
        texts = np.array(texts)
    if type(phrases) != np.ndarray:
        phrases = np.array(phrases)
    if type(targets) != np.ndarray:
        targets = np.array(targets)

    skf.get_n_splits(texts, targets)
    train_dataset = []
    test_dataset = []
    for train_index, test_index in skf.split(texts, targets):
        # print("TRAIN:", train_index, "TEST:", test_index)
        text_train, text_test = texts[train_index], texts[test_index]
        phrase_train, phrase_test = phrases[train_index], phrases[test_index]
        targets_train, targets_test = targets[train_index], targets[test_index]

        train_dataset.append((text_train, phrase_train, targets_train))
        test_dataset.append((text_test, phrase_test, targets_test))

    return train_dataset, test_dataset



In [ ]:
from logging import error
import torch
import numpy as np
from transformers import BertTokenizerFast
from torch.utils.data.dataloader import default_collate
import torch.nn.functional as F
import re
import warnings
from transformers import AutoTokenizer, AutoModelForSequenceClassification


class BERT_Dataset_Type_3:
    def __init__(self,
                 texts,
                 phrases,
                 targets,
                 tokenizer_save_path,
                 tokenizer_method=BertTokenizerFast,
                 max_len=128):
        self.texts = texts
        self.phrases = phrases
        self.targets = targets
        self.tokenizer = tokenizer_method.from_pretrained(
            tokenizer_save_path,
            do_lower_case=True,
        )
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):

        # Calling the indexes of texts and phrases
        text = self.texts[item]
        phrase = self.phrases[item]
        text = " ".join(text.split()).lower()
        phrase = " ".join(phrase.split()).lower()
        tok = self.tokenizer(
            text,
            phrase,
            # pad_to_max_length=True,
            # max_length=self.max_len,
            return_tensors='pt',
            return_token_type_ids="True"
        )
        return tok.input_ids[0], tok.attention_mask[0], tok.token_type_ids[0], torch.tensor(self.targets[item], dtype=torch.float)

class BERT_Dataset_Type_4:
    def __init__(self,
                 texts,
                 phrases,
                 targets,
                 tokenizer_save_path,
                 tokenizer_method=BertTokenizerFast,
                 max_len=128):
        self.texts = texts
        self.phrases = phrases
        self.targets = targets
        self.tokenizer = tokenizer_method.from_pretrained(
            tokenizer_save_path,
            do_lower_case=True,
        )
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):

        # Calling the indexes of texts and phrases
        text = self.texts[item]
        phrase = self.phrases[item]
        text = " ".join(text.split()).lower()
        phrase = " ".join(phrase.split()).lower()
        tok = self.tokenizer(
            text,
            # pad_to_max_length=True,
            # max_length=self.max_len,
            return_tensors='pt',
            return_token_type_ids="True"
        )
        return tok.input_ids[0], tok.attention_mask[0], tok.token_type_ids[0], torch.tensor(self.targets[item], dtype=torch.float)



def custom_collate(data):
    max_len = 0
    ids, masks, tok_types, targets = map(list, zip(*data))
    for id in ids:
        max_len = max(max_len, id.shape[0])
    
    padded_ids, padded_masks, padded_tok_types = [], [], []
    
    for id in ids:
        pad_size = max_len - id.shape[0]
        padded_ids.append(F.pad(id, (0, pad_size), "constant", 0))

    for mask in masks:
        pad_size = max_len - mask.shape[0]
        padded_masks.append(F.pad(mask, (0, pad_size), "constant", 0))

    for tok_type in tok_types:
        pad_size = max_len - tok_type.shape[0]
        padded_tok_types.append(F.pad(tok_type, (0, pad_size), "constant", 0))

    padded_ids = torch.stack(padded_ids, dim=0)
    padded_masks = torch.stack(padded_masks, dim=0)
    padded_tok_types = torch.stack(padded_tok_types, dim=0)
    targets = torch.stack(targets, dim=0)
    return padded_ids, padded_masks, padded_tok_types, targets




In [ ]:

import time
import numpy as np
import torch
import torch.nn as nn
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import pandas as pd


def classify_default(model, tok_ids, attn_mask, tok_type, device):
    tok_ids = tok_ids.to(device)
    attn_mask = attn_mask.to(device)
    tok_type = tok_type.to(device)
    return model(tok_ids,
                 attention_mask=attn_mask,
                 token_type_ids=tok_type,
                 return_dict=False)[0]

def classify_distill(model, tok_ids, attn_mask, tok_type, device):
    tok_ids = tok_ids.to(device)
    attn_mask = attn_mask.to(device)
    tok_type = tok_type.to(device)
    return model(tok_ids,
                 attention_mask=attn_mask,
                #  token_type_ids=tok_type,
                 return_dict=False)[0]


def loss_fn_cross_entropy(outputs, targets):
    return nn.CrossEntropyLoss()(outputs, targets.to(int))


class Trainer:
    """
    Will be used for Trainer Class for training the model

    1. Will train the model
    2. Will evaluate the model
    3. Will save the torch model
    4. Will save the model to onnx
    5. Can connect it to wandb
    """

    def __init__(self,
                 model,
                 classify,
                 config,
                 device=None
                 ):
        if device is not None:
            self.device = device
        else:
            self.device = torch.device(
                "cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.model = model.to(self.device)
        self.optimizer = AdamW(model.parameters(),
                               lr=config["learning_rate"])
        self.criterion = loss_fn_cross_entropy
        self.classify = classify
        self.config = config
        num_train_steps = int(config["df_len"] / config["train_batch_size"] * config["num_epochs"])
        self.scheduler = get_linear_schedule_with_warmup(
            self.optimizer, num_warmup_steps=0,
            num_training_steps=num_train_steps
        )

    def train_classification(self,
                             train_dataloader,
                             val_dataloader,
                             start_step=0):

        training_log = []

        for epoch in range(self.config["num_epochs"]):
            start_time = time.time()
            self.model.train()
            running_loss = []

            for batch_idx, batch in tqdm(enumerate(train_dataloader),
                                         total=len(train_dataloader),
                                         leave=False):
                self.optimizer.zero_grad()
                tok_ids, attn_mask, tok_type, targets = batch

                predictions = self.classify(self.model, tok_ids, attn_mask, tok_type, self.device)
                loss = self.criterion(predictions, targets.to(self.device))
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()
                running_loss.append(loss.item())

                start_step += 1

            running_loss = np.mean(running_loss)
            overall_acc, acc_1, acc_2, acc_3, f1, val_loss = self.get_validation(
                val_dataloader)
            elapsed = time.time() - start_time

            training_log.append([epoch, running_loss, val_loss,
                                 acc_1, acc_2, acc_3, f1])
            print(
                "| epoch {:3d} | train loss {:8.5f} | Val loss {:8.5f} | Overall acc {:5.3f} | "
                "class_1_acc {:5.3f} | class_2_acc {:5.3f} | class_3_acc {:5.3f} | "
                "f1_score {:5.3f} | Time Taken {:5.3f}s |".format(
                    epoch, running_loss, val_loss, overall_acc,
                    acc_1, acc_2, acc_3, f1, elapsed
                )
            )
            print(" ".join("-" for _ in range(75)))

        return training_log

    def get_validation(self, val_dataloader):
        self.model.eval()
        running_loss = []
        y_pred = []
        y_true = []
        with torch.no_grad():
            for batch_idx, batch in enumerate(val_dataloader):
                tok_ids, attn_mask, tok_type, targets = batch

                predictions = self.classify(self.model, tok_ids, attn_mask, tok_type, self.device)
                loss = self.criterion(
                    predictions, targets.to(self.device))
                predictions = torch.nn.functional.softmax(
                    predictions, 1).detach().cpu()
                predictions = torch.argmax(predictions, dim = 1)
                running_loss.append(loss.item())
                y_pred += predictions.tolist()
                y_true += targets.cpu().tolist()

        f1 = f1_score( y_true, y_pred, average="weighted", zero_division=1)
        overall_acc = np.mean(np.array(y_true) == np.array(y_pred))
        matrix = confusion_matrix(y_true, y_pred)
        matrix = matrix.diagonal()/matrix.sum(axis=1)
        return overall_acc, matrix[0], matrix[1], matrix[2], f1, np.mean(running_loss)


# Tiny Bert Cross Validation with 5 folds

## Method-
  1. Given a Sentence and Phrase, we concatenate them with the [SEP] token
  2. Example - 
    - Sentence - "The milk was good"
    - phrase - "milk"
    - Transformer input - "The milk was good [SEP] milk"

## Results - 

  1. Fold 0-
    - Overall Accuracy - 72.9%
    - weighted f1 - 0.72
    - Negative accuracy - 74%
    - Nuetral accuracy - 73%
    - Positive accuracy - 69%

  2. Fold 1-   
    - Overall Accuracy - 73.6%
    - weighted f1 - 0.736
    - Negative accuracy - 75.6%
    - Nuetral accuracy - 73.4%
    - Positive accuracy - 70.7%

  3. Fold 2-   
    - Overall Accuracy - 75.1%
    - weighted f1 - 0.751
    - Negative accuracy - 76.5%
    - Nuetral accuracy - 80.7%
    - Positive accuracy - 65.9%

  4. Fold 3-   
    - Overall Accuracy - 71.9%
    - weighted f1 - 0.718
    - Negative accuracy - 75.9%
    - Nuetral accuracy - 71.0%
    - Positive accuracy - 66.3%

  5. Fold 4-   
    - Overall Accuracy - 71.9%
    - weighted f1 - 0.719
    - Negative accuracy - 71.9%
    - Nuetral accuracy - 72.9%
    - Positive accuracy - 75.2%

---




In [ ]:
config = {
    "splits": 5,
    "seed": 0,
    "train_batch_size": 32,
    "val_batch_size": 64,
    "num_workers": 2,
    "learning_rate": 5e-5,
    "df_len": 3200,
    "num_epochs": 7,
    "max_len": 128,
    "checkpoint": "huawei-noah/TinyBERT_General_4L_312D",
    "classify": classify_default,

}
texts_all = df["clean_text"].to_numpy()
phrases_all = df["clean_aspect"].to_numpy()
targets_all = df["label"].to_numpy()

train_dataset, test_dataset = build_k_datasets(texts_all, 
                                               phrases_all, 
                                               targets_all, 
                                               config["splits"], 
                                               config["seed"])

for i, dataset in enumerate(zip(train_dataset, test_dataset)):
    print(f"Training for fold {i}")
    print(" ".join("=" for _ in range(75)))
    checkpoint = config["checkpoint"]
    train, test = dataset
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 3)
    texts, phrases, targets = train
    val_texts, val_phrases, val_targets = test

    train_dataset = BERT_Dataset_Type_3(texts, phrases, targets, checkpoint, tokenizer_method = AutoTokenizer, max_len=config["max_len"])
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config["train_batch_size"], 
        num_workers=config["num_workers"],
        collate_fn = custom_collate
    )

    val_dataset = BERT_Dataset_Type_3(val_texts, val_phrases, val_targets, checkpoint, tokenizer_method = AutoTokenizer, max_len=config["max_len"])
    val_data_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=config["val_batch_size"],
        num_workers=config["num_workers"],
        collate_fn = custom_collate
    )
    engine = Trainer(model, config["classify"], config)

    log = engine.train_classification(
        train_data_loader,
        val_data_loader
    )


Training for fold 0
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =


Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'fit_denses.4.weight', 'fit_denses.1.bias', 'fit_denses.2.weight', 'cls.predictions.decoder.weight', 'fit_denses.3.weight', 'cls.predictions.bias', 'fit_denses.0.weight', 'fit_denses.2.bias', 'fit_denses.1.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'fit_denses.3.bias', 'fit_denses.0.bias', 'fit_denses.4.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

| epoch   2 | train loss  0.58459 | Val loss  0.71688 | Overall acc 0.716 | class_1_acc 0.777 | class_2_acc 0.620 | class_3_acc 0.738 | f1_score 0.715 | Time Taken 9.545s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   3 | train loss  0.51481 | Val loss  0.69475 | Overall acc 0.723 | class_1_acc 0.714 | class_2_acc 0.767 | class_3_acc 0.680 | f1_score 0.723 | Time Taken 9.499s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   4 | train loss  0.43911 | Val loss  0.72665 | Overall acc 0.725 | class_1_acc 0.702 | class_2_acc 0.798 | class_3_acc 0.670 | f1_score 0.726 | Time Taken 9.542s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   5 | train loss  0.40238 | Val loss  0.72446 | Overall acc 0.724 | class_1_acc 0.717 | class_2_acc 0.787 | class_3_acc 0.655 | f1_score 0.724 | Time Taken 9.507s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   6 | train loss  0.36700 | Val loss  0.72545 | Overall acc 0.719 | class_1_acc 0.729 | class_2_acc 0.752 | class_3_acc 0.660 | f1_score 0.719 | Time Taken 9.507s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


# Bert Cross Validation with 5 folds

## Method-
  1. Given a Sentence and Phrase, we concatenate them with the [SEP] token
  2. Example - 
    - Sentence - "The milk was good"
    - phrase - "milk"
    - Transformer input - "The milk was good [SEP] milk"

## Results - 

  1. Fold 0-
    - Overall Accuracy - 72.3%
    - weighted f1 - 0.72
    - Negative accuracy - 77.8%
    - Nuetral accuracy - 74.9%
    - Positive accuracy - 66.3%

  2. Fold 1-   
    - Overall Accuracy - 73.1%
    - weighted f1 - 0.736
    - Negative accuracy - 74.7%
    - Nuetral accuracy - 72.6%
    - Positive accuracy - 71.2%

  3. Fold 2-   
    - Overall Accuracy - 73.9%
    - weighted f1 - 0.73.9
    - Negative accuracy - 72.3%
    - Nuetral accuracy - 81.5%
    - Positive accuracy - 66.8%

  4. Fold 3-   
    - Overall Accuracy - 71.8%
    - weighted f1 - 0.718
    - Negative accuracy - 72.6%
    - Nuetral accuracy - 76.1%
    - Positive accuracy - 64.9%

  5. Fold 4-   
    - Overall Accuracy - 73.4%
    - weighted f1 - 0.735
    - Negative accuracy - 73.2%
    - Nuetral accuracy - 73.3%
    - Positive accuracy - 73.8%

---




In [ ]:
config = {
    "splits": 5,
    "seed": 0,
    "train_batch_size": 8,
    "val_batch_size": 16,
    "num_workers": 2,
    "learning_rate": 5e-6,
    "df_len": 3200,
    "num_epochs": 5,
    "max_len": 128,
    "checkpoint": "bert-base-uncased",
    "classify": classify_default,

}
texts_all = df["clean_text"].to_numpy()
phrases_all = df["clean_aspect"].to_numpy()
targets_all = df["label"].to_numpy()

train_dataset, test_dataset = build_k_datasets(texts_all, 
                                               phrases_all, 
                                               targets_all, 
                                               config["splits"], 
                                               config["seed"])

for i, dataset in enumerate(zip(train_dataset, test_dataset)):
    print(f"Training for fold {i}")
    print(" ".join("=" for _ in range(75)))
    checkpoint = config["checkpoint"]
    train, test = dataset
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 3)
    texts, phrases, targets = train
    val_texts, val_phrases, val_targets = test

    train_dataset = BERT_Dataset_Type_3(texts, phrases, targets, checkpoint, tokenizer_method = AutoTokenizer, max_len=config["max_len"])
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config["train_batch_size"], 
        num_workers=config["num_workers"],
        collate_fn = custom_collate
    )

    val_dataset = BERT_Dataset_Type_3(val_texts, val_phrases, val_targets, checkpoint, tokenizer_method = AutoTokenizer, max_len=config["max_len"])
    val_data_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=config["val_batch_size"],
        num_workers=config["num_workers"],
        collate_fn = custom_collate
    )
    engine = Trainer(model, config["classify"], config)

    log = engine.train_classification(
        train_data_loader,
        val_data_loader
    )


Training for fold 0
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

...

| epoch   0 | train loss  0.99425 | Val loss  0.81744 | Overall acc 0.649 | class_1_acc 0.643 | class_2_acc 0.756 | class_3_acc 0.524 | f1_score 0.647 | Time Taken 90.842s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   1 | train loss  0.73765 | Val loss  0.68140 | Overall acc 0.708 | class_1_acc 0.685 | class_2_acc 0.798 | class_3_acc 0.631 | f1_score 0.708 | Time Taken 90.530s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   2 | train loss  0.59814 | Val loss  0.65315 | Overall acc 0.733 | class_1_acc 0.759 | class_2_acc 0.736 | class_3_acc 0.684 | f1_score 0.733 | Time Taken 90.466s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   3 | train loss  0.52072 | Val loss  0.63999 | Overall acc 0.729 | class_1_acc 0.723 | class_2_acc 0.721 | class_3_acc 0.748 | f1_score 0.730 | Time Taken 90.498s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   4 | train loss  0.47784 | Val loss  0.63500 | Overall acc 0.734 | class_1_acc 0.732 | class_2_acc 0.733 | class_3_acc 0.738 | f1_score 0.735 | Time Taken 90.626s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


# Tiny Bert Abalation stufy

### Model Used - Tiny BERT

## Method-
  1. Given a Sentence and Phrase, we pass only the sentence without phrase
  2. Example - 
    - Sentence - "The milk was good"
    - phrase - "milk"
    - Transformer input - "The milk was good"

## Results - 

  1. Fold 0-
    - Overall Accuracy - 67.1%
    - weighted f1 - 0.671
    - Negative accuracy - 73.8%
    - Nuetral accuracy - 60.2%
    - Positive accuracy - 64.9%

  2. Fold 1-   
    - Overall Accuracy - 71.8%
    - weighted f1 - 0.716
    - Negative accuracy - 77.7%
    - Nuetral accuracy - 71.0%
    - Positive accuracy - 62.9%

  3. Fold 2-
    - Overall Accuracy - 70.8%
    - weighted f1 - 0.708
    - Negative accuracy - 75.9%
    - Nuetral accuracy - 69.9%
    - Positive accuracy - 63.4%

  4. Fold 3-   
    - Overall Accuracy - 66.1%
    - weighted f1 - 0.731
    - Negative accuracy - 73.8%
    - Nuetral accuracy - 61.0%
    - Positive accuracy - 60.0%

  5. Fold 4-
    - Overall Accuracy - 78.6%
    - weighted f1 - 0.716
    - Negative accuracy - 72.6%
    - Nuetral accuracy - 69.8%
    - Positive accuracy - 60.7%

---




In [ ]:
config = {
    "splits": 5,
    "seed": 0,
    "train_batch_size": 32,
    "val_batch_size": 64,
    "num_workers": 2,
    "learning_rate": 5e-5,
    "df_len": 3200,
    "num_epochs": 7,
    "max_len": 128,
    "checkpoint": "huawei-noah/TinyBERT_General_4L_312D",
    "classify": classify_default,

}
texts_all = df["clean_text"].to_numpy()
phrases_all = df["clean_aspect"].to_numpy()
targets_all = df["label"].to_numpy()

train_dataset, test_dataset = build_k_datasets(texts_all, 
                                               phrases_all, 
                                               targets_all, 
                                               config["splits"], 
                                               config["seed"])

for i, dataset in enumerate(zip(train_dataset, test_dataset)):
    print(f"Training for fold {i}")
    print(" ".join("=" for _ in range(75)))
    checkpoint = config["checkpoint"]
    train, test = dataset
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 3)
    texts, phrases, targets = train
    val_texts, val_phrases, val_targets = test

    train_dataset = BERT_Dataset_Type_4(texts, phrases, targets, checkpoint, tokenizer_method = AutoTokenizer, max_len=config["max_len"])
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config["train_batch_size"], 
        num_workers=config["num_workers"],
        collate_fn = custom_collate
    )

    val_dataset = BERT_Dataset_Type_4(val_texts, val_phrases, val_targets, checkpoint, tokenizer_method = AutoTokenizer, max_len=config["max_len"])
    val_data_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=config["val_batch_size"],
        num_workers=config["num_workers"],
        collate_fn = custom_collate
    )
    engine = Trainer(model, config["classify"], config)

    log = engine.train_classification(
        train_data_loader,
        val_data_loader
    )


Training for fold 0
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =


Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'fit_denses.4.weight', 'fit_denses.1.bias', 'fit_denses.2.weight', 'cls.predictions.decoder.weight', 'fit_denses.3.weight', 'cls.predictions.bias', 'fit_denses.0.weight', 'fit_denses.2.bias', 'fit_denses.1.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'fit_denses.3.bias', 'fit_denses.0.bias', 'fit_denses.4.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a

| epoch   0 | train loss  1.06620 | Val loss  1.01442 | Overall acc 0.531 | class_1_acc 0.795 | class_2_acc 0.548 | class_3_acc 0.078 | f1_score 0.476 | Time Taken 9.169s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   1 | train loss  0.95178 | Val loss  0.92040 | Overall acc 0.635 | class_1_acc 0.622 | class_2_acc 0.637 | class_3_acc 0.654 | f1_score 0.636 | Time Taken 9.084s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   2 | train loss  0.85615 | Val loss  0.86754 | Overall acc 0.641 | class_1_acc 0.616 | class_2_acc 0.645 | class_3_acc 0.678 | f1_score 0.642 | Time Taken 9.037s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


...

| epoch   2 | train loss  0.86534 | Val loss  0.84170 | Overall acc 0.654 | class_1_acc 0.631 | class_2_acc 0.640 | class_3_acc 0.709 | f1_score 0.655 | Time Taken 9.397s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   3 | train loss  0.81254 | Val loss  0.80083 | Overall acc 0.675 | class_1_acc 0.685 | class_2_acc 0.659 | class_3_acc 0.680 | f1_score 0.676 | Time Taken 9.383s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   4 | train loss  0.77477 | Val loss  0.78017 | Overall acc 0.679 | class_1_acc 0.705 | class_2_acc 0.667 | class_3_acc 0.650 | f1_score 0.679 | Time Taken 9.409s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   5 | train loss  0.75680 | Val loss  0.77244 | Overall acc 0.676 | class_1_acc 0.699 | class_2_acc 0.674 | class_3_acc 0.641 | f1_score 0.677 | Time Taken 9.345s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   6 | train loss  0.74598 | Val loss  0.76615 | Overall acc 0.686 | class_1_acc 0.726 | class_2_acc 0.698 | class_3_acc 0.607 | f1_score 0.686 | Time Taken 9.372s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


# Bert Abalation stufy

### Model Used - BERT

## Method-
  1. Given a Sentence and Phrase, we pass only the sentence without phrase
  2. Example - 
    - Sentence - "The milk was good"
    - phrase - "milk"
    - Transformer input - "The milk was good"

## Results - 

  1. Fold 0-
    - Overall Accuracy - 71.6%
    - weighted f1 - 0.716
    - Negative accuracy - 73.2%
    - Nuetral accuracy - 72.6%
    - Positive accuracy - 67.8%

  2. Fold 1-   
    - Overall Accuracy - 73.1%
    - weighted f1 - 0.731
    - Negative accuracy - 78.0%
    - Nuetral accuracy - 71.0%
    - Positive accuracy - 67.8%

---




In [ ]:
config = {
    "splits": 5,
    "seed": 0,
    "train_batch_size": 8,
    "val_batch_size": 16,
    "num_workers": 2,
    "learning_rate": 5e-6,
    "df_len": 3200,
    "num_epochs": 5,
    "max_len": 128,
    "checkpoint": "bert-base-uncased",
    "classify": classify_default,

}
texts_all = df["clean_text"].to_numpy()
phrases_all = df["clean_aspect"].to_numpy()
targets_all = df["label"].to_numpy()

train_dataset, test_dataset = build_k_datasets(texts_all, 
                                               phrases_all, 
                                               targets_all, 
                                               config["splits"], 
                                               config["seed"])

for i, dataset in enumerate(zip(train_dataset, test_dataset)):
    if i is 2:
      break
    print(f"Training for fold {i}")
    print(" ".join("=" for _ in range(75)))
    checkpoint = config["checkpoint"]
    train, test = dataset
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 3)
    texts, phrases, targets = train
    val_texts, val_phrases, val_targets = test

    train_dataset = BERT_Dataset_Type_4(texts, phrases, targets, checkpoint, tokenizer_method = AutoTokenizer, max_len=config["max_len"])
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config["train_batch_size"], 
        num_workers=config["num_workers"],
        collate_fn = custom_collate
    )

    val_dataset = BERT_Dataset_Type_4(val_texts, val_phrases, val_targets, checkpoint, tokenizer_method = AutoTokenizer, max_len=config["max_len"])
    val_data_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=config["val_batch_size"],
        num_workers=config["num_workers"],
        collate_fn = custom_collate
    )
    engine = Trainer(model, config["classify"], config)

    log = engine.train_classification(
        train_data_loader,
        val_data_loader
    )


Training for fold 0
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

| epoch   0 | train loss  0.84823 | Val loss  0.74270 | Overall acc 0.676 | class_1_acc 0.705 | class_2_acc 0.683 | class_3_acc 0.620 | f1_score 0.677 | Time Taken 88.201s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   1 | train loss  0.61707 | Val loss  0.71509 | Overall acc 0.690 | class_1_acc 0.696 | class_2_acc 0.734 | class_3_acc 0.624 | f1_score 0.691 | Time Taken 87.954s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   2 | train loss  0.48465 | Val loss  0.72053 | Overall acc 0.704 | class_1_acc 0.705 | class_2_acc 0.745 | class_3_acc 0.649 | f1_score 0.704 | Time Taken 87.925s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


...

| epoch   0 | train loss  0.88104 | Val loss  0.71377 | Overall acc 0.701 | class_1_acc 0.771 | class_2_acc 0.761 | class_3_acc 0.512 | f1_score 0.697 | Time Taken 87.740s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   1 | train loss  0.62512 | Val loss  0.69315 | Overall acc 0.728 | class_1_acc 0.795 | class_2_acc 0.757 | class_3_acc 0.580 | f1_score 0.725 | Time Taken 87.607s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   2 | train loss  0.49614 | Val loss  0.70292 | Overall acc 0.733 | class_1_acc 0.807 | class_2_acc 0.757 | class_3_acc 0.580 | f1_score 0.730 | Time Taken 87.630s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   3 | train loss  0.38487 | Val loss  0.73478 | Overall acc 0.734 | class_1_acc 0.801 | class_2_acc 0.703 | class_3_acc 0.663 | f1_score 0.733 | Time Taken 87.682s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


| epoch   4 | train loss  0.32873 | Val loss  0.75203 | Overall acc 0.731 | class_1_acc 0.780 | class_2_acc 0.710 | class_3_acc 0.678 | f1_score 0.731 | Time Taken 87.678s |
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
